In [9]:
import os

# Change the current working directory to the specified path
new_directory = r'C:\Users\vladimir.shpilkin\Desktop\Learning\Python-Projects\py_sql_analysis'
os.chdir(new_directory)

# Print the new current working directory
print('New directory is', os.getcwd())

# Import necessary libraries
import sqlite3
import pandas as pd

# Read the CSV file located in the new directory
bank_data = 'Employee_Salary_Dataset.csv'
bank_data_df = pd.read_csv(bank_data)

# Establish a connection to the SQLite database
connection_employee_salary = sqlite3.connect('Employee_Salary_Dataset.db')

# Write the DataFrame to the SQLite database
employee_salary = bank_data_df.to_sql('Employee_Salary_Dataset', connection_employee_salary, if_exists='replace', index=False)

# Create a cursor for the database
cursor = connection_employee_salary.cursor()


New directory is c:\Users\vladimir.shpilkin\Desktop\Learning\Python-Projects\py_sql_analysis


In [10]:
#find the average salary for male workers
results_male = cursor.execute('SELECT "Age", ROUND(AVG("Salary"), 1) AS "Avg_Male_Salary" FROM "Employee_Salary_Dataset" WHERE "Gender" = "Male"')
avg_male_salary = results_male.fetchone()[1]
print(avg_male_salary)

2063626.5


In [11]:
#find the average salary for female workers
results_female = cursor.execute('SELECT "Age", ROUND(AVG("Salary"), 1) AS "Avg_Male_Salary" FROM "Employee_Salary_Dataset" WHERE "Gender" = "Female"')
avg_female_salary = results_male.fetchone()[1]
print(avg_female_salary)

2054916.7


In [12]:
#find the difference
round(avg_male_salary-avg_female_salary,2)

8709.8

In [13]:
#find the average salary per age group and gender
age_bins=cursor.execute('''
                            SELECT CASE WHEN "Age" BETWEEN 18 and 24 then "18-24"
                            WHEN "Age" BETWEEN 25 and 34 then "25-34"
                            WHEN "Age" BETWEEN 35 and 44 then "35-44"
                            WHEN "Age" BETWEEN 45 and 54 then "45-54"
                            WHEN "Age" BETWEEN 55 and 65 then "55-65"
                            WHEN "Age" <18 then "underage"
                            WHEN "Age" >65 then "retirement"
                            END AS age_bin, round(avg("Salary"),1), "Gender",
                            case when Gender='Male' then count(ID) else 0 END AS male_count,
                            case when Gender='Female' then COUNT(ID) else 0 end as female_count,
                            round(avg(Experience_Years))
                            from Employee_Salary_Dataset
                            GROUP BY age_bin, "Gender"
                            '''
                            )

In [14]:
#display the avg. salary per gender and age bin
age_bins.fetchall()

[('18-24', 36666.7, 'Female', 0, 6, 3.0),
 ('18-24', 20100.0, 'Male', 5, 0, 2.0),
 ('25-34', 379250.0, 'Female', 0, 4, 6.0),
 ('25-34', 2087500.0, 'Male', 4, 0, 6.0),
 ('35-44', 61500.0, 'Female', 0, 1, 10.0),
 ('35-44', 2183383.3, 'Male', 3, 0, 12.0),
 ('45-54', 4730000.0, 'Female', 0, 5, 17.0),
 ('45-54', 5023333.3, 'Male', 3, 0, 15.0),
 ('55-65', 5770000.0, 'Female', 0, 2, 24.0),
 ('55-65', 5001000.0, 'Male', 1, 0, 25.0),
 ('underage', 10000.0, 'Male', 1, 0, 1.0)]

In [15]:
#increase the salary for all males in the age group between 18 and 24
salary_update=cursor.execute('''
                            UPDATE Employee_Salary_Dataset
                            SET Salary=
                            CASE WHEN Gender='Male'
                            and Age>=18 and Age<=24 then salary +3000
                            when Gender='Female'
                            and Age>=25 and Age<=34 then salary +5000
                            end
                            ''')
salary_update.fetchall()
connection_employee_salary.commit()


In [24]:
check = cursor.execute('''
    SELECT *
    FROM Employee_Salary_Dataset
    WHERE (Age >= 18 AND Age <= 24 AND Gender = 'Male')
       OR (Age >= 25 AND Age <= 34 AND Gender = 'Female')
''')
cursor.fetchall()

[(1, 5, 28, 'Female', 255000),
 (2, 1, 21, 'Male', 53000),
 (4, 2, 22, 'Male', 28000),
 (11, 4, 26, 'Female', 255000),
 (15, 2, 23, 'Male', 10500),
 (16, 4, 27, 'Female', 92000),
 (17, 10, 34, 'Female', 935000),
 (19, 2, 21, 'Male', 18000),
 (27, 1, 18, 'Male', 6000)]